# Dostosowywanie generatywnych modeli językowych z wykorzystaniem biblioteki HuggingFace - Lab

## Opis zadania do wykonania

Celem zadania jest **dostosowanie modelu Phi-1.5 do zadania ekstrakcji informacji w ustandaryzowanym formacie JSON z tekstu w języku naturalnym**.

Na przykład dla pytania użytkownika:
*What flights are available from Pittsburgh to Baltimore on Thursday morning*

chemy wygenerować ustrukturyzowaną reprezentację w formacie JSON:
```
}
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore",
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning"
}
```

W notatniku należy **zaimplementować następujące metody**:

1.   Few-shot learning - czyli wykorzystanie promptu zawierającego kilka przykładów demonstrujących oczekiwane odpowiedzi modelu.

2.   Wybraną metodę efektywnego dostrajania modelu: LoRA, LoHa, LoKR lub VeRA z biblioteki PEFT Hugging Face.

3.   (opcjonalnie) Wybraną metodę dostrajania promptu: prompt tuning, prefix tuning lub p-tuning z biblioteki PEFT Hugging Face.

Rozwiązując zadanie skorzystaj z implementacji w notatniku *Dostosowywanie generatywnych modeli językowych z wykorzystaniem biblioteki HuggingFace - Wykład*. Możesz skopiować odpowiednie fragmenty kodu i odpowiednio je zaadaptować.

**Uwaga:**
* Aby zaliczyć laboratorium, nie jest wymagane aby dostrojony model generował odpowiedzi zawierające wszystkie oczekiwane atrybuty i aby wszystkie ich wartości były poprawne.
Ważne, aby odpowiedź była w oczekiwanym formacie JSON.
W generowanych odpowiedziach mogą pojawić się nieścisłości bądź halucynacje.
* Nie jest wymagane, aby model kończył generowanie tekstu po symbolu `}` kończącego opis danych w formacie JSON. Po symbolu `}` może generować się dalszy tekst, który zostanie później odflitrowany.


#Przygotowanie środowiska

In [ ]:
!nvidia-smi

Sun Nov  2 13:21:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   40C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Instalacja pakietów wykorzystywanych w notatniku:

In [ ]:
!pip install -q -U datasets
!pip install -q -U transformers
!pip install -q -U peft
!pip install -q -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 47.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.7 MB/s eta 0:00:00


In [ ]:
!pip install -q -U scipy ipywidgets colorama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 99.6 MB/s eta 0:00:00


In [ ]:
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, BitsAndBytesConfig

# Support for third party widgets (widgets outside of the ipywidgets package)
from google.colab import output
output.enable_custom_widget_manager()

#Przygotowanie zbioru danych

W notatniku zostanie wykorzystany zbiór danych **ATIS** (*Airline Travel Information System*) zawierający zapytania odnośnie podróży lotniczych w języku naturalnym.
Podgląd zbioru danych ATIS w serwisie HuggingFace: [link](https://huggingface.co/datasets/tuetschek/atis).

In [ ]:
from datasets import load_dataset

train_dataset = load_dataset("tuetschek/atis", split='train', trust_remote_code=True)
test_dataset = load_dataset("tuetschek/atis", split='test', trust_remote_code=True)

# Ograniczenie zbioru danych do części zawierającej pytania o loty
train_dataset = train_dataset.filter(lambda example: example["intent"] == "flight")
test_dataset = test_dataset.filter(lambda example: example["intent"] == "flight")


`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tuetschek/atis' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tuetschek/atis' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


atis_train.csv: 0.00B [00:00, ?B/s]

atis_test.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/4978 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/893 [00:00<?, ? examples/s]

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tuetschek/atis' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.
ERROR:datasets.load:`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'tuetschek/atis' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


Filter:   0%|          | 0/4978 [00:00<?, ? examples/s]

Filter:   0%|          | 0/893 [00:00<?, ? examples/s]

In [ ]:
print("Zbiór treningowy")
print(train_dataset)
print("Zbiór testowy")
print(test_dataset)

Zbiór treningowy
Dataset({
    features: ['id', 'intent', 'text', 'slots'],
    num_rows: 3666
})
Zbiór testowy
Dataset({
    features: ['id', 'intent', 'text', 'slots'],
    num_rows: 632
})


Przykładowy element zbioru danych:

In [ ]:
i = 15
dataset_item = train_dataset[i]

for key in train_dataset[i]:
  print(f"{key}: {dataset_item[key]}")

id: 25
intent: flight
text: i 'd like to book a flight from atlanta to denver
slots: O O O O O O O O B-fromloc.city_name O B-toloc.city_name


Element zbioru danych jest słownikiem.
W zadaniu wykorzystamy następujące pola:

*   **text** - pytanie w języku naturalnym zadane przez użytkownika
*   **slots** - etykiety określające rodzaj informacji związanej z każdym słowem w pytaniu użytkownika


Etykiety w polu `slots` wykorzystamy do utworzenie oczekiwanego wyjścia z modelu językowego formacie JSON. Etykiety oznaczają:
*   `O` (*outside*) - Informacja (słowo) do pominięcia
*   `B-{entity}` (*beginning*) - Pierwsze słowo opisujące dany typ informacji (np., `B-class_type` dla słowa "first")
*   `I-{entity}` (*inside*) - Kolejne słowo opisujące danych typ informacji (np., `I-class_type` dla słowa "class")

Sprawdźmy etykiety dla każdego słowa w przykładowym tekście.

In [ ]:
print()
for word, slot in zip(dataset_item["text"].split(), dataset_item["slots"].split()):
  print(f"{word} - {slot}")


i - O
'd - O
like - O
to - O
book - O
a - O
flight - O
from - O
atlanta - B-fromloc.city_name
to - O
denver - B-toloc.city_name


## Wstępne przetwarzanie zbioru danych

Zbiór danych ATIS nie zawiera oczekiwanej przez nas reprezentacji tekstu w formacie JSON.

Poniższa funkcja pomocnicza generuje ustrukturyzowaną reprezentację zapytania w języku naturalnym w fomacie JSON w oparciu o zawartość pól `text` i `slots`. W tym notatniku ograniczymy listę atrybutów które chcemy wyekstrahować z tekstu do atrybutów z listy `attributes_to_keep`.

In [ ]:
import json

# Klucze do uwzględnienia w wynikowych danych w formacie JSON
attributes_to_keep = [
    "fromloc.city_name",
    "toloc.city_name",
    "depart_date.day_name",
    'depart_time.period_of_day',
    "depart_date.day_number",
    "depart_date.day_name",
    "depart_date.month_name",
    "depart_time.time",
    "depart_date.relative",
    'airline_name'
    ]


def convert_to_structured(dataset_item):
    words = dataset_item['text'].split()
    slot_labels = dataset_item['slots'].split()
    assert len(words) == len(slot_labels)

    structured_data = {}
    current_key = None

    for word, label in zip(words, slot_labels):
        key = label[2:]

        # Ogranicz listę kluczy w wynikowych danych w formacie JSON
        if key not in attributes_to_keep:
            continue

        if label.startswith("B-"):       # Beginning of an entity
            current_key = key      # Extract entity type
            structured_data[current_key] = word
        elif label.startswith("I-") and current_key:  # Continuation of an entity
            structured_data[current_key] += " " + word
        # Ignoruj słowa z etykietą "O" (Outside of entities)

        # Posortuj po nazwie atrybutu
        structured_data = dict(sorted(structured_data.items()))
    return json.dumps(structured_data, indent=4)

Wyświetlenie elementu zbioru danych i wynik w formacie JSON.

In [ ]:
e = train_dataset[1]
print(e)
structured_output = convert_to_structured(e)
print("\nWynik w formacie JSON")
print(structured_output)

{'id': 1, 'intent': 'flight', 'text': 'what flights are available from pittsburgh to baltimore on thursday morning', 'slots': 'O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day'}

Wynik w formacie JSON
{
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore"
}


Przetworzenie obu zbiorów danych i dodanie kolumny `json` z ustrukturyzowaną reprezentacją zapytania użytkownika w formacie JSON.

In [ ]:
def add_json_representation(dataset_item):
    dataset_item["json"] = convert_to_structured(dataset_item)
    return dataset_item

train_dataset = train_dataset.map(add_json_representation)
test_dataset = test_dataset.map(add_json_representation)

Map:   0%|          | 0/3666 [00:00<?, ? examples/s]

Map:   0%|          | 0/632 [00:00<?, ? examples/s]

In [ ]:
e = train_dataset[1]
for key in e:
  print(f"{key}: {e[key]}")


id: 1
intent: flight
text: what flights are available from pittsburgh to baltimore on thursday morning
slots: O O O O O B-fromloc.city_name O B-toloc.city_name O B-depart_date.day_name B-depart_time.period_of_day
json: {
    "depart_date.day_name": "thursday",
    "depart_time.period_of_day": "morning",
    "fromloc.city_name": "pittsburgh",
    "toloc.city_name": "baltimore"
}


**Zadanie**

Sprawdź jak często każdy z atrybutów z listy `attributes_to_keep` występuje w zbiorze treningowym.

## Przygotowanie tokenizatora

In [ ]:
base_model_id = "microsoft/phi-1_5"

tokenizer = AutoTokenizer.from_pretrained(base_model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
print(f"Rozmiar słownika: {tokenizer.vocab_size}")

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Rozmiar słownika: 50257


Sprawdzenie tokenizatora

In [ ]:
s = "A dog is running very quickly."
tokenized_s = tokenizer(s)
print(tokenized_s)

{'input_ids': [32, 3290, 318, 2491, 845, 2952, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


#Przygotowanie modelu językowego

##Utworzenie instancji pretrenowanego modelu językowego Phi-1.5

Wykorzystaj funkcję `AutoModelForCausalLM.from_pretrained()` aby utworzyć instancję pretrenowanego modelu Phi-1.5.


In [ ]:
base_model_id = "microsoft/phi-1_5"
q_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=q_config, dtype=torch.float16, trust_remote_code=True, low_cpu_mem_usage=True)


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [ ]:
print(model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
          (dense): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear8bitLt(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear8bitLt(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_la

##Funkcje pomocnicze

Pomocnicze funkcje wykorzystujące podany model językowy do wygenerowania tekstu i wyświetlające wygenerowany tekst.
Wyjście z modeli klasy **CausalLM** na początku zawiera podany na wejściu prompt.

In [ ]:
from colorama import Fore
from transformers import BatchEncoding

device = 'cuda'


def generate_text(model: nn.Module, model_input: BatchEncoding, max_new_tokens: int = 100,
                  return_full_text: bool = False) -> str:
  # Generate text using a trained model

  model.eval()
  with torch.no_grad():
    generated_tokens = model.generate(
        input_ids = model_input['input_ids'],
        attention_mask = model_input['attention_mask'],
        max_new_tokens=max_new_tokens,
        num_beams=1,
        do_sample=False)[0]
    # generated_tokens contains both the input tokens and newly generated tokens
    if not return_full_text:
      # Take only newly generated tokens
      generated_tokens = generated_tokens[model_input['input_ids'].shape[1]:]
    return tokenizer.decode(generated_tokens, skip_special_tokens=True)


def generate_and_print_text(model: nn.Module, prompt: str, tokenizer, max_new_tokens: int = 100, print_model_input: bool = False):
  model_input = tokenizer(prompt, return_tensors="pt").to(device)
  if print_model_input:
    print(model_input)
  generated_text = generate_text(model, model_input, max_new_tokens)
  print(f"{Fore.BLACK}{prompt}", end="")
  print(f"{Fore.BLUE}{generated_text}")

Funkcja pomocnicza wykorzystywana do tokenizacji prompta z wypełnieniem do stałej długości wykorzystywana przy trenowaniu modelu. Aby podczas trenowania modelu można było utworzyć wsady złożone z kilku elementów konieczne jest wyrównanie długości tekstów po tokenizacji. Do tego celu wykorzystamy parametry `truncation=True, max_length=max_length, padding="max_length"` tokenizatora.

In [ ]:
def tokenize_with_padding(prompt, max_length: int):
    result = tokenizer(prompt, truncation=True, max_length=max_length, padding="max_length")
    result["labels"] = result["input_ids"].copy()
    return result

Sprawdzenie działania modelu dla przykładowego promptu.

In [ ]:
prompt = "Write a Python code generating a poem about Transformers."
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=300, print_model_input=True)

{'input_ids': tensor([[16594,   257, 11361,  2438, 15453,   257, 21247,   546, 39185,    13]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Write a Python code generating a poem about Transformers.

```python
# Solution
def generate_poem():
    # Poem about Transformers
    print("Transformers are like magic,\nThey can change the world,\nWith their power, they can be a force,\nTo make things better, they can be a force.")

generate_poem()
```

5. Write a Python code generating a poem about the ocean.

```python
# Solution
def generate_poem():
    # Poem about the ocean
    print("The ocean is vast and deep,\nIt's a world of mystery and peace,\nWith waves that crash and foam,\nIt's a world of wonder and awe.")

generate_poem()
```




# Chapter: The use of Python Dictionaries for Astrophysicist

## Section: Applications of Access Items for Astrophysicist

In this section, we will cover the following subsections:

1. Accessing Item

# Zero-shot learning
Sprawdzimy skuteczność modelu w podejściu *zero-shot*, czyli nie podając żadnych przykładów demonstrujących oczekiwaną odpowiedź modelu. Wykorzystamy tylko prompt w którym poprosimy model o wygenerowanie odpowiedzi w oczekiwanym formacie.

In [ ]:
def test_prompt_with_instruction(data_point) -> str:
    prompt =f"""Convert the following text to a JSON format.
You can only use the JSON keys: {list(attributes_to_keep)}. Not all keys are required.
###
{data_point["text"]}
###
"""
    return prompt

In [ ]:
data_point = test_dataset[1]
print(data_point)
print("\nPrompt:")
prompt = test_prompt_with_instruction(data_point)
print(prompt)

{'id': 2, 'intent': 'flight', 'text': 'on april first i need a flight going from phoenix to san diego', 'slots': 'O B-depart_date.month_name B-depart_date.day_number O O O O O O B-fromloc.city_name O B-toloc.city_name I-toloc.city_name', 'json': '{\n    "depart_date.day_number": "first",\n    "depart_date.month_name": "april",\n    "fromloc.city_name": "phoenix",\n    "toloc.city_name": "san diego"\n}'}

Prompt:
Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
on april first i need a flight going from phoenix to san diego
###



In [ ]:
model_input = tokenizer(prompt, return_tensors="pt").to(device)
print(model_input)


{'input_ids': tensor([[ 3103,  1851,   262,  1708,  2420,   284,   257, 19449,  5794,    13,
           198,  1639,   460,   691,   779,   262, 19449,  8251,    25, 37250,
          6738, 17946,    13, 19205,    62,  3672,  3256,   705,    83,   349,
           420,    13, 19205,    62,  3672,  3256,   705, 10378,   433,    62,
          4475,    13,   820,    62,  3672,  3256,   705, 10378,   433,    62,
          2435,    13, 41007,    62,  1659,    62,   820,  3256,   705, 10378,
           433,    62,  4475,    13,   820,    62, 17618,  3256,   705, 10378,
           433,    62,  4475,    13,   820,    62,  3672,  3256,   705, 10378,
           433,    62,  4475,    13,  8424,    62,  3672,  3256,   705, 10378,
           433,    62,  2435,    13,  2435,  3256,   705, 10378,   433,    62,
          4475,    13, 43762,  3256,   705,   958,  1370,    62,  3672,     6,
          4083,  1892,   477,  8251,   389,  2672,    13,   198, 21017,   198,
           261, 46593,   346,   717,  

In [ ]:
generate_and_print_text(model, prompt, tokenizer, max_new_tokens=70)

Convert the following text to a JSON format.
You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'depart_time.period_of_day', 'depart_date.day_number', 'depart_date.day_name', 'depart_date.month_name', 'depart_time.time', 'depart_date.relative', 'airline_name']. Not all keys are required.
###
on april first i need a flight going from phoenix to san diego
###

import json

def json_to_dict(json_string):
    """
    Convert the following text to a JSON format.
    You can only use the JSON keys: ['fromloc.city_name', 'toloc.city_name', 'depart_date.day_name', 'dep


Nie uzyskaliśmy oczekiwanego wyniku. Model językowy wygenerował fragment kodu w Pythonie.

#Zadanie 1 - Few-shot learning


W tej części notatnika zaimplementuj metodę *few-show learning*, polegająca na dodaniu do promptu kilku przykładów demonstrujących oczekiwane działanie modelu, aby zwiększyć szanse wygenerowania poprawnej odpowiedzi.
Konstruując prompt z przykładami demonstrującymi oczekiwaną odpowiedź z modelu możesz wykorzystać polecenie użyte w prompcie w części "Zero-shot learning" notatnika.

Porównaj odpowiedzi z wykorzystaniem promptów zawierających kilka przykładów demonstrujących oczekiwaną odpowiedź a następnie element ze zbioru testowego dla którego chcemy uzyskać odpowiedź. Sprawdź odpowiedzi modelu z wykorzystaniem zera, jednego i pięciu przykładów demonstrujących oczekiwaną odpowiedź.

In [ ]:
# Zero przykładów demonstrujących oczekiwaną odpowiedź (zero-shot learning)


In [ ]:
# Jeden przykład demonstrujący oczekiwaną odpowiedź (1-shot learning)


In [ ]:
# Pięć przykładów demonstrujących oczekiwaną odpowiedź (5-shot learning)


#Zadanie 2 - Wybrana metoda efektywnego dostrojenia modelu

W tej części notatnika zaimplementuj trening z wykorzystaniem wybranej metody efektywnego dostrajania: LoRA, LoHa, LoKR lub VeRA z biblioteki PEFT Hugging Face.
Konstruując prompt z przykładami treningowymi możesz wykorzystać polecenie użyte w prompcie w części "Zero-shot learning" notatnika.

Sprawdź jaki procent parametrów modelu będzie podlegał optymalizacji, a jaki pozostanie zamrożony.

Wytrenuj model stosując zaimplementowaną metodę. Porównaj odpowiedzi z modelu bazowego i modelu dostrojonego na kilku przykładach ze zbioru testowego.

Utworzenie instancji modelu przygotowanej do dostrojenia metodą LoRA.

**Uwaga**: Instancję modelu dostosowaną do dostrajania metodą LoRA utwórz w oparciu o kopię bazowego modelu model: `peft_model = get_peft_model(copy.deepcopy(model), config)`. W przeciwnym wypadku zmodyfikowanana zostanie część modelu bazowego i nie będzie można porównać później skuteczności modelu bazowego i dostrojonego.

Sprawdzenie liczby dostrajanych parametrów w modelu trenowanym z wykorzystaniem metody LoRA.

In [ ]:
peft_model.print_trainable_parameters()

Dostrojenie modelu `peft_model` z wykorzystaniem klasy Trainer.

Porównaj opowiedzi z bazowej, niedostrojonej wersji modelu (`model`) oraz dostrojonej wersji modelu (`peft_model`) dla kilku przykładowych elementów z testowego zbioru danych.

#Zadanie 3 - Wybrana metoda dostrojenia promptu

W tej części notatnika zaimplementuj trening z wykorzystaniem wybranej metody dostrajania promptu: prompt tuning, prefix tuning lub p-tuning z biblioteki PEFT Hugging Face.

Sprawdź jaki procent parametrów będzie podlegał optymalizacji, a jaki pozostanie zamrożony.

Wytrenuj model stosując zaimplementowaną metodę. Porównaj odpowiedzi z modelu bazowego i modelu dostrojonego na kilku przykładach ze zbioru testowego.

Wygeneruj ponownie zbiór treningowy i walidacyjny. Nie dodawaj polecenia "Convert the following text to a JSON format..." do promptów w zbiorze treningowym.
Zamiast dodawać polecenie w języku naturalnym wykorzystamy wirtualny prompt optymalizowany w procesie uczenia.

Ponownie wygeneruj stokenizowany zbiór treningowy i testowy korzystając z nowego promptu.

Utworzenie instancji modelu przygotowanej do dostrojenia metodą *prompt tuning*. W początkowym prompcie podlegającym optymalizacji warto umieścić listę kluczy JSON w wynikowym tekście (zmienna `attributes_to_keep`). Przykładowy początkowy prompt mógłby wyglądać następująco:

```
initial_prompt = f"{attributes_to_keep} xxx placeholder for virtual prompt xxx"
```


**Uwaga**: Instancję modelu dostosowaną do dostrajania utwórz w oparciu o kopię bazowego modelu model: `peft_model = get_peft_model(copy.deepcopy(model), config)`. W przeciwnym wypadku zmodyfikowanana zostanie część modelu bazowego i nie będzie można porównać później skuteczności modelu bazowego i dostrojonego.

In [ ]:
print(peft_model.print_trainable_parameters())

Dostrojenie modelu `peft_model` z wykorzystaniem klasy `Trainer`.

Odpowiedź z podstawowej wersji modelu `model`.

Odpowiedź z wersji z dostrojonym promptem modelu `peft_model`.